In [ ]:
import pandas as pd

data = pd.read_csv('movies_metadata.csv', low_memory=False)

data['genres'] = data['genres'].apply(eval)

# Split the 'genres' column into individual rows using explode, expanding each dictionary within the list into its own separate row.
data = data.explode('genres')

# Filter out non-dictionary values
data = data[data['genres'].apply(lambda x: isinstance(x, dict))]

# Convert the dictionaries into separate columns
data[['genre_id', 'genre_name']] = pd.DataFrame(data['genres'].tolist())

data.drop(columns=['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'], inplace=True)

# Resetting the index
data.reset_index(drop=True, inplace=True)

# Renaming the columns
data.rename(columns={'genre_id': 'genre_id',
                     'genre_name': 'genre_name',
                     'id': 'movie_id'}, inplace=True)

regex_pattern = r"^\d{4}-\d{2}-\d{2}$"

# Creating a boolean mask to filter rows
mask = ~data['movie_id'].str.contains(regex_pattern)

# Drop rows using `.drop` with the mask (in-place)
data.drop(data[~mask].index, inplace=True)

data['genres_comp'] = data['movie_id'].astype(str) + '_' + data['genre_id'].astype(str)

# Converting movie_id datatype from str to int
data['movie_id'] = data['movie_id'].astype(int)

# Dropping primary keys duplicates
data.drop_duplicates(subset='genres_comp', keep='first', inplace = True)

# Save the Data into a new file
data.to_csv('final_genres.csv', index = False)

int64
